# Jakost gibanja plosce s tipali SensorTile.box

## Pregled ali se v Orange Data Mining izvede predprocesiranje podatkov pred klasifikacijo s klasifikacijskim drevesom (V tem primeru se ne izvaja predprocesiranje podatkov) 

In [15]:
import pickle

with open('../../Orange_Data_Mining/Jakost_gibanja/Klasifikacijsko_drevo-Jakost_gibanja.pkcls', 'rb') as model:
    lr = pickle.load(model)

#lr.domain
lr.domain[0]

ContinuousVariable(name='F1_VAR_on_ACC_V', number_of_decimals=3)

## Prikaz klasifikacijskega modela iz Orange Data Mining

In [1]:
import pickle

with open('../../Orange_Data_Mining/Jakost_gibanja/Klasifikacijsko_drevo-Jakost_gibanja.pkcls', 'rb') as model:
    lr = pickle.load(model)#pickle.loads(model)

printed_tree = lr.print_tree()
for i in printed_tree.split('\n'):
    print(i)

f:\FRI_novSSD\Orange\lib\site-packages\pyqtgraph\colors\palette.py:1: RuntimeWarning: PyQtGraph supports Qt version >= 5.15, but 5.12.9 detected.
  from ..Qt import QtGui


       [60.  0.  0.] F1_VAR_on_ACC_V ≤ 7.60756e-07
       [ 0. 62. 62.] F1_VAR_on_ACC_V > 7.60756e-07
       [ 0. 61.  4.]     F1_VAR_on_ACC_V ≤ 0.120615
       [ 0. 51.  0.]         F1_VAR_on_ACC_V ≤ 0.0457891
       [ 0. 10.  4.]         F1_VAR_on_ACC_V > 0.0457891
       [ 0.  1. 58.]     F1_VAR_on_ACC_V > 0.120615



## Pretvorba zapisa klasifikacijskega drevesa iz Orange Data Mining v obliko, ki se jo lahko uporabi v programu Unico GUI (zapis se shrani v datoteko txt)

In [3]:
import pickle

# pickleFile -> pickle file with classification tree
# newTxtFile -> new txt file with classification tree to be used in Unico-GUI
# classNames -> list of class names for values to be classified to (order is not important as it gets sorted alphabetically)
def orangeToUnicoTree(pickleFile, newTxtFile, classNames):
    orderedClassNames = classNames.copy()
    orderedClassNames.sort()

    with open(pickleFile, 'rb') as model:
        lr = pickle.load(model)#pickle.loads(model)

    printed_tree = lr.print_tree()
    splited_tree = printed_tree.strip().split('\n')
    new_line = ""
    numberOfLeaves = 0
    numberOfNodes = 1 #0

    for i in range(len(splited_tree)):
        splited_tree[i] = splited_tree[i].strip()

    with open(newTxtFile, 'w') as f:
        f.write("\n")

        for i in range(len(splited_tree)):
            new_line = ""
            if(i+1 < len(splited_tree)):
                numberOfCurrRowIndents = 0
                numberOfNextRowIndents = 0

                start = splited_tree[i].find("]")
                start += 2
                for j in range(start, len(splited_tree[i]), 4):
                    if(splited_tree[i][j] == " "):
                        new_line += "|   "
                        numberOfCurrRowIndents += 1
                    else:
                        break
                new_line += splited_tree[i][start+numberOfCurrRowIndents*4:]

                startNext = splited_tree[i+1].find("]")
                startNext += 2
                for j in range(startNext, len(splited_tree[i+1]), 4):
                    if(splited_tree[i+1][j] == " "):
                        numberOfNextRowIndents += 1
                    else:
                        break         

                if(not (numberOfCurrRowIndents < numberOfNextRowIndents)): #check indent
                    numberOfLeaves += 1
                    classValues = splited_tree[i][1:start-3].split(".")
                    classValues = [int(value) for value in classValues] #turn to integers to compare numbers
                    max_index = classValues.index(max(classValues))
                    new_line += ": " + orderedClassNames[max_index]
                    numberOfNodes += 2


            else: #Last line is leaf node
                numberOfCurrRowIndents = 0

                start = splited_tree[i].find("]")
                start += 2
                for j in range(start, len(splited_tree[i]), 4):
                    if(splited_tree[i][j] == " "):
                        new_line += "|   "
                        numberOfCurrRowIndents += 1
                    else:
                        break
                new_line += splited_tree[i][start+numberOfCurrRowIndents*4:]

                classValues = splited_tree[i][1:start-3].split(".")
                max_index = classValues.index(max(classValues))
                new_line += ": " + orderedClassNames[max_index]

                numberOfLeaves += 1

            new_line = new_line.replace("≤", "<=")
            f.write(new_line + "\n")

        f.write("\n")
        f.write("Number of Leaves  : 	 " + str(numberOfLeaves) + "\n")
        f.write("\n")
        f.write("Size of the Tree : 	 " + str(numberOfNodes) + "\n")


class_names = ["mirovanje", "srednja_jakost", "velika_jakost"]
drevo = '../../Orange_Data_Mining/Jakost_gibanja/Klasifikacijsko_drevo-Jakost_gibanja.pkcls'
datotekaZNovimZapisom = 'DrevoJakostGibanjaTXT-Orange.txt'

orangeToUnicoTree(drevo, datotekaZNovimZapisom, class_names)

## Sprememba klasifikacijskega drevesa iz .txt datoteke v if-else stavke, ki se lahko izvedejo v programski kodi Python (programska koda se shrani v datoteko txt)

### Klasifikacijsko drevo iz Orange Data Mining

In [4]:
# treeTxtFile -> txt file with classification tree to be used in Unico-GUI
# newTxtFile -> new txt file with with classification tree turned to If-Else statements in Python code
def txtTreeToIfElseStatements(treeTxtFile, newTxtFile):
    with open(treeTxtFile, 'r') as model:
        drevo = model.read()

    vrstice = drevo.strip().split('\n')


    with open(newTxtFile, 'w') as f:
        f.write("import pandas as pd" + "\n")
        f.write("dfIzhod = []" + "\n")
        f.write("\n")
        f.write("def klasifikacija(vhodnaVrednost): \n")

        stevecZamakov = -1
        konecIzvajanja = False

        for i in range(len(vrstice)):

            if(konecIzvajanja):
                break

            ifStavek = ""
            besede = vrstice[i].split()
            for j in range(len(besede)):
                if besede[j][0] == '<' or besede[j][0] == '>':

                    f.write("\t")
                    f.write("\t" * (j-1))

                    if(j-1 <= stevecZamakov):
                        f.write("el")
                        stevecZamakov = j-1
                    elif(j-1 > stevecZamakov):
                        stevecZamakov = j-1

                    if j+2 < len(besede):
                        f.write( "if vhodnaVrednost[\"" + besede[j-1] + "\"] " + besede[j] + " " + besede[j+1] + "\n")
                        f.write("\t" * (j+1) + "dfIzhod.append(\"" + besede[j+2] + "\") " + "\n")
                    else:
                        f.write( "if vhodnaVrednost[\"" + besede[j-1] + "\"] " + besede[j] + " " + besede[j+1] + ": \n")
                    break

                elif len(besede[j]) == 1 and besede[j][0] == ':':
                    konecIzvajanja = True
                    break

txtTreeToIfElseStatements('DrevoJakostGibanjaTXT-Orange.txt', 'DrevoJakostGibanja-IF_ELSE-Orange.txt')


### Klasifikacijsko drevo iz Unico-GUI

In [5]:
potDoUnicoDrevesa = '../../AlgoBuilder/Klasifikacijsko_drevo-Jakost_gibanja/ST_decision_tree_20230607_113100_235.txt'
txtTreeToIfElseStatements(potDoUnicoDrevesa, 'DrevoJakostGibanja-IF_ELSE-Unico.txt')


## Uporaba if-else stavkov klasifikacijskega drevesa za izracun klasifikacijske tocnosti in mere F1

### Klasifikacijsko drevo iz Orange Data Mining

In [6]:
import pandas as pd
dfIzhod = []

def klasifikacija(vhodnaVrednost): 
	if vhodnaVrednost["F1_VAR_on_ACC_V"] <= 7.60756e-07:
		dfIzhod.append("mirovanje") 
	elif vhodnaVrednost["F1_VAR_on_ACC_V"] > 7.60756e-07: 
		if vhodnaVrednost["F1_VAR_on_ACC_V"] <= 0.120615: 
			if vhodnaVrednost["F1_VAR_on_ACC_V"] <= 0.0457891:
				dfIzhod.append("srednja_jakost") 
			elif vhodnaVrednost["F1_VAR_on_ACC_V"] > 0.0457891:
				dfIzhod.append("srednja_jakost") 
		elif vhodnaVrednost["F1_VAR_on_ACC_V"] > 0.120615:
			dfIzhod.append("velika_jakost") 

df = pd.read_csv('../../Zajeti_podatki/Jakost_gibanja/Testni_podatki/zdruzeni_testni_podatki-varianca.csv')

df.apply(lambda row : klasifikacija(row), axis = 1)

from sklearn.metrics import accuracy_score, f1_score

print("Klasifikacijska tocnost: " + str(accuracy_score(df["class"].values.tolist(), dfIzhod)))
print("Mera F1: " + str(f1_score(df["class"].values.tolist(), dfIzhod, average='weighted')))


Klasifikacijska tocnost: 0.875
Mera F1: 0.8752770657729528


### Klasifikacijsko drevo iz Unico-GUI

In [7]:
import pandas as pd
dfIzhod = []

def klasifikacija(vhodnaVrednost): 
	if vhodnaVrednost["F1_VAR_on_ACC_V"] <= 0.050293: 
		if vhodnaVrednost["F1_VAR_on_ACC_V"] <= 0.00585938: 
			if vhodnaVrednost["F1_VAR_on_ACC_V"] <= 0.000976563: 
				if vhodnaVrednost["F1_VAR_on_ACC_V"] <= 0:
					dfIzhod.append("srednja_jakost") 
				elif vhodnaVrednost["F1_VAR_on_ACC_V"] > 0:
					dfIzhod.append("mirovanje") 
			elif vhodnaVrednost["F1_VAR_on_ACC_V"] > 0.000976563: 
				if vhodnaVrednost["F1_VAR_on_ACC_V"] <= 0.00390625:
					dfIzhod.append("mirovanje") 
				elif vhodnaVrednost["F1_VAR_on_ACC_V"] > 0.00390625: 
					if vhodnaVrednost["F1_VAR_on_ACC_V"] <= 0.00488281:
						dfIzhod.append("srednja_jakost") 
					elif vhodnaVrednost["F1_VAR_on_ACC_V"] > 0.00488281:
						dfIzhod.append("mirovanje") 
		elif vhodnaVrednost["F1_VAR_on_ACC_V"] > 0.00585938:
			dfIzhod.append("srednja_jakost") 
	elif vhodnaVrednost["F1_VAR_on_ACC_V"] > 0.050293: 
		if vhodnaVrednost["F1_VAR_on_ACC_V"] <= 0.118164:
			dfIzhod.append("srednja_jakost") 
		elif vhodnaVrednost["F1_VAR_on_ACC_V"] > 0.118164:
			dfIzhod.append("velika_jakost") 

df = pd.read_csv('../../Zajeti_podatki/Jakost_gibanja/Testni_podatki/zdruzeni_testni_podatki-varianca.csv')

df.apply(lambda row : klasifikacija(row), axis = 1)

from sklearn.metrics import accuracy_score, f1_score

print("Klasifikacijska tocnost: " + str(accuracy_score(df["class"].values.tolist(), dfIzhod)))
print("Mera F1: " + str(f1_score(df["class"].values.tolist(), dfIzhod, average='weighted')))


Klasifikacijska tocnost: 0.8152173913043478
Mera F1: 0.8099008047944183


---

# Hoja oseb

## Pregled ali se v Orange Data Mining izvede predprocesiranje podatkov pred klasifikacijo s klasifikacijskim drevesom (V tem primeru se ne izvaja predprocesiranje podatkov nad nobenim od 16ih atributov) 

In [38]:
import pickle

with open('../../Orange_Data_Mining/Hoja_oseb/Klasifikacijsko_drevo-Hoja_oseb.pkcls', 'rb') as model:
    lr = pickle.load(model)

#LOCENO PREVERJANJE VSAKEGA ATRIBUTA (ukaz print() ne izpise predprocesiranja podatkov za posamezni atribut)
#lr.domain
#lr.domain[0]
#lr.domain[1]
#lr.domain[2]
#lr.domain[3]
#lr.domain[4]
#lr.domain[5]
#lr.domain[6]
#lr.domain[7]
#lr.domain[8]
#lr.domain[9]
#lr.domain[10]
#lr.domain[11]
#lr.domain[12]
#lr.domain[13]
#lr.domain[14]
lr.domain[15]

ContinuousVariable(name='F16_MAX_on_ACC_V', number_of_decimals=3)

## Prikaz klasifikacijskega modela iz Orange Data Mining

In [8]:
import pickle

with open('../../Orange_Data_Mining/Hoja_oseb/Klasifikacijsko_drevo-Hoja_oseb.pkcls', 'rb') as model:
    lr = pickle.load(model)

printed_tree = lr.print_tree()
for i in printed_tree.split('\n'):
    print(i)

[ 14. 302.  51. 303. 307.] F2_MEAN_on_ACC_Y ≤ 0.00819231
[ 14. 260.  51.   8. 280.]     F8_VAR_on_ACC_V ≤ 0.131636
[  2.   3.  51.   5. 204.]         F10_MIN_on_ACC_Y ≤ -0.84
[  1.   3.   0.   5. 204.]             F13_MAX_on_ACC_X ≤ -0.163
[ 1.  0. 51.  0.  0.]             F13_MAX_on_ACC_X > -0.163
[ 12. 257.   0.   3.  76.]         F10_MIN_on_ACC_Y > -0.84
[ 7. 18.  0.  0. 64.]             F3_MEAN_on_ACC_Z ≤ -0.045375
[ 7. 10.  0.  0.  1.]                 F9_MIN_on_ACC_X ≤ -1.657
[ 0.  8.  0.  0. 63.]                 F9_MIN_on_ACC_X > -1.657
[ 0.  1.  0.  0. 46.]                     F4_MEAN_on_ACC_V ≤ 1.04024
[ 0.  7.  0.  0. 17.]                     F4_MEAN_on_ACC_V > 1.04024
    [0. 6. 0. 0. 5.]                         F14_MAX_on_ACC_Y ≤ 0.428
[ 0.  1.  0.  0. 12.]                         F14_MAX_on_ACC_Y > 0.428
[  5. 239.   0.   3.  12.]             F3_MEAN_on_ACC_Z > -0.045375
[ 0. 13.  0.  0. 10.]                 F2_MEAN_on_ACC_Y ≤ -0.131106
[ 0. 10.  0.  0.  0.]                

## Pretvorba zapisa klasifikacijskega drevesa iz Orange Data Mining v obliko, ki se jo lahko uporabi v programu Unico GUI (zapis se shrani v datoteko txt)

In [9]:
class_names = ["Mirovanje", "Jaz", "Sestra", "Vati", "Mami"]
drevo = '../../Orange_Data_Mining/Hoja_oseb/Klasifikacijsko_drevo-Hoja_oseb.pkcls'
datotekaZNovimZapisom = 'DrevoHojaOsebTXT-Orange.txt'

orangeToUnicoTree(drevo, datotekaZNovimZapisom, class_names)

## Sprememba klasifikacijskega drevesa iz .txt datoteke v if-else stavke, ki se lahko izvedejo v programski kodi Python (programska koda se shrani v datoteko txt)

### Klasifikacijsko drevo iz Orange Data Mining

In [10]:
txtTreeToIfElseStatements('DrevoHojaOsebTXT-Orange.txt', 'DrevoHojaOseb-IF_ELSE-Orange.txt')

### Klasifikacijsko drevo iz Unico-GUI

In [11]:
potDoUnicoDrevesa = '../../AlgoBuilder/Klasifikacijsko_drevo-Hoja_oseb/ST_decision_tree_20230626_160707_340.txt'
txtTreeToIfElseStatements(potDoUnicoDrevesa, 'DrevoHojaOseb-IF_ELSE-Unico.txt')

## Uporaba if-else stavkov klasifikacijskega drevesa za izracun klasifikacijske tocnosti in mere F1

### Klasifikacijsko drevo iz Orange Data Mining

In [12]:
import pandas as pd
dfIzhod = []

def klasifikacija(vhodnaVrednost): 
	if vhodnaVrednost["F2_MEAN_on_ACC_Y"] <= 0.00819231: 
		if vhodnaVrednost["F8_VAR_on_ACC_V"] <= 0.131636: 
			if vhodnaVrednost["F10_MIN_on_ACC_Y"] <= -0.84: 
				if vhodnaVrednost["F13_MAX_on_ACC_X"] <= -0.163:
					dfIzhod.append("Vati") 
				elif vhodnaVrednost["F13_MAX_on_ACC_X"] > -0.163:
					dfIzhod.append("Mirovanje") 
			elif vhodnaVrednost["F10_MIN_on_ACC_Y"] > -0.84: 
				if vhodnaVrednost["F3_MEAN_on_ACC_Z"] <= -0.045375: 
					if vhodnaVrednost["F9_MIN_on_ACC_X"] <= -1.657:
						dfIzhod.append("Mami") 
					elif vhodnaVrednost["F9_MIN_on_ACC_X"] > -1.657: 
						if vhodnaVrednost["F4_MEAN_on_ACC_V"] <= 1.04024:
							dfIzhod.append("Vati") 
						elif vhodnaVrednost["F4_MEAN_on_ACC_V"] > 1.04024: 
							if vhodnaVrednost["F14_MAX_on_ACC_Y"] <= 0.428:
								dfIzhod.append("Mami") 
							elif vhodnaVrednost["F14_MAX_on_ACC_Y"] > 0.428:
								dfIzhod.append("Vati") 
				elif vhodnaVrednost["F3_MEAN_on_ACC_Z"] > -0.045375: 
					if vhodnaVrednost["F2_MEAN_on_ACC_Y"] <= -0.131106: 
						if vhodnaVrednost["F11_MIN_on_ACC_Z"] <= -0.773:
							dfIzhod.append("Mami") 
						elif vhodnaVrednost["F11_MIN_on_ACC_Z"] > -0.773:
							dfIzhod.append("Vati") 
					elif vhodnaVrednost["F2_MEAN_on_ACC_Y"] > -0.131106:
						dfIzhod.append("Mami") 
		elif vhodnaVrednost["F8_VAR_on_ACC_V"] > 0.131636: 
			if vhodnaVrednost["F6_VAR_on_ACC_Y"] <= 0.0872378: 
				if vhodnaVrednost["F2_MEAN_on_ACC_Y"] <= -0.132202: 
					if vhodnaVrednost["F11_MIN_on_ACC_Z"] <= -0.551:
						dfIzhod.append("Vati") 
					elif vhodnaVrednost["F11_MIN_on_ACC_Z"] > -0.551:
						dfIzhod.append("Vati") 
				elif vhodnaVrednost["F2_MEAN_on_ACC_Y"] > -0.132202: 
					if vhodnaVrednost["F15_MAX_on_ACC_Z"] <= 0.991:
						dfIzhod.append("Mami") 
					elif vhodnaVrednost["F15_MAX_on_ACC_Z"] > 0.991:
						dfIzhod.append("Mami") 
			elif vhodnaVrednost["F6_VAR_on_ACC_Y"] > 0.0872378:
				dfIzhod.append("Sestra") 
	elif vhodnaVrednost["F2_MEAN_on_ACC_Y"] > 0.00819231: 
		if vhodnaVrednost["F6_VAR_on_ACC_Y"] <= 1.62777e-05:
			dfIzhod.append("Mirovanje") 
		elif vhodnaVrednost["F6_VAR_on_ACC_Y"] > 1.62777e-05:
			dfIzhod.append("Jaz") 


df = pd.read_csv('../../Zajeti_podatki/Hoja_oseb/Testni_podatki/Unico_zdruzeni_testni_podatki-KlasifikacijskoDrevo.csv')

df.apply(lambda row : klasifikacija(row), axis = 1)

from sklearn.metrics import accuracy_score, f1_score

print("Klasifikacijska tocnost: " + str(accuracy_score(df["class"].values.tolist(), dfIzhod)))
print("Mera F1: " + str(f1_score(df["class"].values.tolist(), dfIzhod, average='weighted')))

Klasifikacijska tocnost: 0.8864229765013055
Mera F1: 0.8862326175387447


### Klasifikacijsko drevo iz Unico-GUI

In [13]:
import pandas as pd
dfIzhod = []

def klasifikacija(vhodnaVrednost): 
	if vhodnaVrednost["F2_MEAN_on_ACC_Y"] <= 0.00840322: 
		if vhodnaVrednost["F8_VAR_on_ACC_V"] <= 0.130886: 
			if vhodnaVrednost["F10_MIN_on_ACC_Y"] <= -0.839918: 
				if vhodnaVrednost["F13_MAX_on_ACC_X"] <= -0.298938: 
					if vhodnaVrednost["F5_VAR_on_ACC_X"] <= 0.0664063:
						dfIzhod.append("Vati") 
					elif vhodnaVrednost["F5_VAR_on_ACC_X"] > 0.0664063: 
						if vhodnaVrednost["F7_VAR_on_ACC_Z"] <= 0.0620117: 
							if vhodnaVrednost["F1_MEAN_on_ACC_X"] <= -1.0127:
								dfIzhod.append("Jaz") 
							elif vhodnaVrednost["F1_MEAN_on_ACC_X"] > -1.0127:
								dfIzhod.append("Vati") 
						elif vhodnaVrednost["F7_VAR_on_ACC_Z"] > 0.0620117: 
							if vhodnaVrednost["F9_MIN_on_ACC_X"] <= -1.64355:
								dfIzhod.append("Sestra") 
							elif vhodnaVrednost["F9_MIN_on_ACC_X"] > -1.64355:
								dfIzhod.append("Mami") 
				elif vhodnaVrednost["F13_MAX_on_ACC_X"] > -0.298938: 
					if vhodnaVrednost["F1_MEAN_on_ACC_X"] <= -0.851563: 
						if vhodnaVrednost["F1_MEAN_on_ACC_X"] <= -1.00391: 
							if vhodnaVrednost["F1_MEAN_on_ACC_X"] <= -1.01074:
								dfIzhod.append("Vati") 
							elif vhodnaVrednost["F1_MEAN_on_ACC_X"] > -1.01074:
								dfIzhod.append("Jaz") 
						elif vhodnaVrednost["F1_MEAN_on_ACC_X"] > -1.00391:
							dfIzhod.append("Mami") 
					elif vhodnaVrednost["F1_MEAN_on_ACC_X"] > -0.851563:
						dfIzhod.append("Mirovanje") 
			elif vhodnaVrednost["F10_MIN_on_ACC_Y"] > -0.839918: 
				if vhodnaVrednost["F3_MEAN_on_ACC_Z"] <= -0.0453391: 
					if vhodnaVrednost["F9_MIN_on_ACC_X"] <= -1.65625: 
						if vhodnaVrednost["F14_MAX_on_ACC_Y"] <= 0.449951: 
							if vhodnaVrednost["F1_MEAN_on_ACC_X"] <= -1.08789:
								dfIzhod.append("Vati") 
							elif vhodnaVrednost["F1_MEAN_on_ACC_X"] > -1.08789:
								dfIzhod.append("Mami") 
						elif vhodnaVrednost["F14_MAX_on_ACC_Y"] > 0.449951: 
							if vhodnaVrednost["F1_MEAN_on_ACC_X"] <= -0.959473:
								dfIzhod.append("Jaz") 
							elif vhodnaVrednost["F1_MEAN_on_ACC_X"] > -0.959473:
								dfIzhod.append("Mami") 
					elif vhodnaVrednost["F9_MIN_on_ACC_X"] > -1.65625: 
						if vhodnaVrednost["F1_MEAN_on_ACC_X"] <= -1.02051:
							dfIzhod.append("Mami") 
						elif vhodnaVrednost["F1_MEAN_on_ACC_X"] > -1.02051:
							dfIzhod.append("Vati") 
				elif vhodnaVrednost["F3_MEAN_on_ACC_Z"] > -0.0453391:
					dfIzhod.append("Mami") 
		elif vhodnaVrednost["F8_VAR_on_ACC_V"] > 0.130886: 
			if vhodnaVrednost["F6_VAR_on_ACC_Y"] <= 0.0873474: 
				if vhodnaVrednost["F2_MEAN_on_ACC_Y"] <= -0.132202: 
					if vhodnaVrednost["F7_VAR_on_ACC_Z"] <= 0.115051: 
						if vhodnaVrednost["F1_MEAN_on_ACC_X"] <= -0.949219:
							dfIzhod.append("Vati") 
						elif vhodnaVrednost["F1_MEAN_on_ACC_X"] > -0.949219: 
							if vhodnaVrednost["F1_MEAN_on_ACC_X"] <= -0.914551:
								dfIzhod.append("Mami") 
							elif vhodnaVrednost["F1_MEAN_on_ACC_X"] > -0.914551:
								dfIzhod.append("Vati") 
					elif vhodnaVrednost["F7_VAR_on_ACC_Z"] > 0.115051: 
						if vhodnaVrednost["F6_VAR_on_ACC_Y"] <= 0.0690308:
							dfIzhod.append("Mami") 
						elif vhodnaVrednost["F6_VAR_on_ACC_Y"] > 0.0690308:
							dfIzhod.append("Sestra") 
				elif vhodnaVrednost["F2_MEAN_on_ACC_Y"] > -0.132202: 
					if vhodnaVrednost["F15_MAX_on_ACC_Z"] <= 1.21582: 
						if vhodnaVrednost["F1_MEAN_on_ACC_X"] <= -0.964355:
							dfIzhod.append("Mami") 
						elif vhodnaVrednost["F1_MEAN_on_ACC_X"] > -0.964355: 
							if vhodnaVrednost["F2_MEAN_on_ACC_Y"] <= -0.128174:
								dfIzhod.append("Mami") 
							elif vhodnaVrednost["F2_MEAN_on_ACC_Y"] > -0.128174:
								dfIzhod.append("Sestra") 
					elif vhodnaVrednost["F15_MAX_on_ACC_Z"] > 1.21582: 
						if vhodnaVrednost["F2_MEAN_on_ACC_Y"] <= -0.110779:
							dfIzhod.append("Mami") 
						elif vhodnaVrednost["F2_MEAN_on_ACC_Y"] > -0.110779:
							dfIzhod.append("Sestra") 
			elif vhodnaVrednost["F6_VAR_on_ACC_Y"] > 0.0873474: 
				if vhodnaVrednost["F13_MAX_on_ACC_X"] <= -0.505696: 
					if vhodnaVrednost["F6_VAR_on_ACC_Y"] <= 0.11261:
						dfIzhod.append("Sestra") 
					elif vhodnaVrednost["F6_VAR_on_ACC_Y"] > 0.11261:
						dfIzhod.append("Vati") 
				elif vhodnaVrednost["F13_MAX_on_ACC_X"] > -0.505696: 
					if vhodnaVrednost["F14_MAX_on_ACC_Y"] <= 0.259033:
						dfIzhod.append("Vati") 
					elif vhodnaVrednost["F14_MAX_on_ACC_Y"] > 0.259033:
						dfIzhod.append("Sestra") 
	elif vhodnaVrednost["F2_MEAN_on_ACC_Y"] > 0.00840322: 
		if vhodnaVrednost["F6_VAR_on_ACC_Y"] <= 6.81505e-005:
			dfIzhod.append("Mirovanje") 
		elif vhodnaVrednost["F6_VAR_on_ACC_Y"] > 6.81505e-005:
			dfIzhod.append("Jaz") 


df = pd.read_csv('../../Zajeti_podatki/Hoja_oseb/Testni_podatki/Unico_zdruzeni_testni_podatki-KlasifikacijskoDrevo.csv')

df.apply(lambda row : klasifikacija(row), axis = 1)

from sklearn.metrics import accuracy_score, f1_score

print("Klasifikacijska tocnost: " + str(accuracy_score(df["class"].values.tolist(), dfIzhod)))
print("Mera F1: " + str(f1_score(df["class"].values.tolist(), dfIzhod, average='weighted')))


Klasifikacijska tocnost: 0.8994778067885117
Mera F1: 0.8984998081595348
